### Abrir navegador, aceptar cookies e introducir código postal

In [2]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from IPython.display import display
from datetime import datetime


# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 2. Abrimos web mercadona
url_mercadona ="https://tienda.mercadona.es/categories/"
driver.get(url_mercadona)

# Esperamos unos segundillos
time.sleep(3)

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"ui-button.ui-button--small.ui-button--primary.ui-button--positive")
cookies.send_keys(Keys.ENTER)

# Buscamos por clase el campo de input para poder introducir nuestro código postal
input = driver.find_element(By.CLASS_NAME,"ym-hide-content")

# Introducimos el código postal
input.send_keys("28016")

# Esperamos unos segundillos
time.sleep(round(random.uniform(0.5,3),1))

# Le damos a ENTER
input.send_keys(Keys.ENTER)   #driver.find_element_by_id("submit").click()



time.sleep(5)

### Recorrer web 2 categorías de momento

In [9]:
dict_mercadona = {"Group":[],"Category":[],"Name":[],"Size":[],"Price":[], "Image":[]}

categories_length = len(driver.find_elements(By.CLASS_NAME,"category-menu__item"))+1
# categories_range = range(1,categories_length)
categories_range = range(1,2)

categories = [] # Títulos de las categorías
sub_header_headers= [] # Títulos de los sub-headers
sub_category= []
empty =[]

category_contador = 0
group_contador = 0
sub_category_contador = 0

def get_product_details(categories,empty,group_contador,category_contador):
    # ONCE IN PRODUCT DISPLAY => RECORRER CADA PRODUCTO Y PINCHAR EN ÉL
    contenedor = driver.find_element(By.CLASS_NAME,"category-detail__content")
    contenedores_grupo = contenedor.find_elements(By.CLASS_NAME,"product-container")

    list_products = []
    list_empty = []
    
    for contenedor in contenedores_grupo:
        
        product_container = contenedor.find_elements(By.CLASS_NAME,"product-cell")
        
        for product in product_container:
            product_image = contenedor.find_element(By.TAG_NAME,"img").get_attribute("src")
            product_array = product.text.split("\n")[:-1]
            product_array.append(product_image)
            
            # ADD ALL PRODUCT INFO TO EMPTY ARRAY
            dict_mercadona["Name"].append(product_array[0])
            dict_mercadona["Size"].append(product_array[1])
            dict_mercadona["Price"].append(product_array[2])
            dict_mercadona["Image"].append(product_array[-1])
            dict_mercadona["Group"].append(categories[group_contador])
            dict_mercadona["Category"].append(empty[category_contador])
            

            list_empty.append(product_array)

        list_products.append(list_empty)
        list_empty = []


for num in categories_range:
    
    # Nivel 1 => GROUP:
    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li:nth-child({num}) > div > button")    
    categories.append(header_a_pinchar[0].text)
    header_a_pinchar[0].click()
    time.sleep(1)

    # Nivel 2 => CATEGORÍAS:
    sub_category_length = len(driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")[0].text.split("\n"))+1
    number_of_sub_headers = range(1,sub_category_length)


    # Pincha todos los botones sub headers y almacena los títulos
    for i in number_of_sub_headers:

        sub_header_button = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li:nth-child({i}) > button")        
        empty.append(sub_header_button[0].text)
        sub_header_button[0].click()
        time.sleep(1)
        # Una vez en el bloque adecuado cogemos todos los productos
        get_product_details(categories,empty,group_contador,category_contador)
        category_contador+=1
        sub_category_contador+=1

    group_contador+=1
    
    # sub_header_headers.append(empty)
    # empty =[]
     


df_mercadona = pd.DataFrame(dict_mercadona)

df_mercadona



# Add columns: Name and Date 
df_mercadona['Shop'] = 'Mercadona' 
today = pd.Timestamp.now()
df_mercadona['Date'] = today 
print(today)
df_mercadona.head()

2023-06-21 19:12:57.431059


,Group,Category,Name,Size,Price,Image,Shop,Date
0,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva 0,4º Hacendado",Garrafa 5 L,"26,25 € /ud.",https://prod-mercadona.imgix.net/images/f89ea1...,Mercadona,2023-06-21 19:12:57.431059
1,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva 0,4º Hacendado",Botella 1 L,"5,40 € /ud.",https://prod-mercadona.imgix.net/images/f89ea1...,Mercadona,2023-06-21 19:12:57.431059
2,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Garrafa 3 L,"19,10 € /ud.",https://prod-mercadona.imgix.net/images/f89ea1...,Mercadona,2023-06-21 19:12:57.431059
3,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Botella 1 L,"6,45 € /ud.",https://prod-mercadona.imgix.net/images/f89ea1...,Mercadona,2023-06-21 19:12:57.431059
4,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado Gran Se...,Botella 750 ml,"5,85 € /ud.",https://prod-mercadona.imgix.net/images/f89ea1...,Mercadona,2023-06-21 19:12:57.431059


In [1]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from IPython.display import display
from datetime import datetime


# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 2. Abrimos web mercadona
url_mercadona ="https://tienda.mercadona.es/categories/"
driver.get(url_mercadona)

# Esperamos unos segundillos
time.sleep(3)

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"ui-button.ui-button--small.ui-button--primary.ui-button--positive")
cookies.send_keys(Keys.ENTER)

# Buscamos por clase el campo de input para poder introducir nuestro código postal
input = driver.find_element(By.CLASS_NAME,"ym-hide-content")

# Introducimos el código postal
input.send_keys("28016")

# Esperamos unos segundillos
time.sleep(round(random.uniform(0.5,3),1))

# Le damos a ENTER
input.send_keys(Keys.ENTER)   #driver.find_element_by_id("submit").click()



time.sleep(5)



dict_mercadona = {"Group":[],"Category":[],"Name":[],"Size":[],"Price":[], "Image":[]}

categories_length = len(driver.find_elements(By.CLASS_NAME,"category-menu__item"))+1
categories_range = range(1,categories_length)

categories = [] # Títulos de las categorías
sub_header_headers= [] # Títulos de los sub-headers
sub_category= []
empty =[]

category_contador = 0
group_contador = 0
sub_category_contador = 0

def get_product_details(categories,empty,group_contador,category_contador):
    # ONCE IN PRODUCT DISPLAY => RECORRER CADA PRODUCTO Y PINCHAR EN ÉL
    contenedor = driver.find_element(By.CLASS_NAME,"category-detail__content")
    contenedores_grupo = contenedor.find_elements(By.CLASS_NAME,"product-container")

    list_products = []
    list_empty = []
    
    for contenedor in contenedores_grupo:
        
        product_container = contenedor.find_elements(By.CLASS_NAME,"product-cell")
        
        for product in product_container:
            product_image = contenedor.find_element(By.TAG_NAME,"img").get_attribute("src")
            product_array = product.text.split("\n")[:-1]
            product_array.append(product_image)
            
            # ADD ALL PRODUCT INFO TO EMPTY ARRAY
            dict_mercadona["Name"].append(product_array[0])
            dict_mercadona["Size"].append(product_array[1])
            dict_mercadona["Price"].append(product_array[2])
            dict_mercadona["Image"].append(product_array[-1])
            dict_mercadona["Group"].append(categories[group_contador])
            dict_mercadona["Category"].append(empty[category_contador])
            

            list_empty.append(product_array)

        list_products.append(list_empty)
        list_empty = []


for num in categories_range:
    
    # Nivel 1 => GROUP:
    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li:nth-child({num}) > div > button")    
    categories.append(header_a_pinchar[0].text)
    header_a_pinchar[0].click()
    time.sleep(1)

    # Nivel 2 => CATEGORÍAS:
    sub_category_length = len(driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")[0].text.split("\n"))+1
    number_of_sub_headers = range(1,sub_category_length)


    # Pincha todos los botones sub headers y almacena los títulos
    for i in number_of_sub_headers:

        sub_header_button = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li:nth-child({i}) > button")        
        empty.append(sub_header_button[0].text)
        sub_header_button[0].click()
        time.sleep(1)
        # Una vez en el bloque adecuado cogemos todos los productos
        get_product_details(categories,empty,group_contador,category_contador)
        category_contador+=1
        sub_category_contador+=1

    group_contador+=1
    
    # sub_header_headers.append(empty)
    # empty =[]
     


df_mercadona = pd.DataFrame(dict_mercadona)

df_mercadona



# Add columns: Name and Date 
df_mercadona['Shop'] = 'Mercadona' 
today = pd.Timestamp.now()
df_mercadona['Date'] = today 
print(today)
df_mercadona.head()



[WDM] - Downloading: 100%|██████████| 8.29M/8.29M [00:00<00:00, 27.6MB/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".category-detail__content"}
  (Session info: chrome=114.0.5735.133)
Stacktrace:
0   chromedriver                        0x0000000102ca16b8 chromedriver + 4937400
1   chromedriver                        0x0000000102c98b73 chromedriver + 4901747
2   chromedriver                        0x0000000102856616 chromedriver + 435734
3   chromedriver                        0x0000000102899e0f chromedriver + 712207
4   chromedriver                        0x000000010289a0a1 chromedriver + 712865
5   chromedriver                        0x00000001028db9a4 chromedriver + 981412
6   chromedriver                        0x00000001028be03d chromedriver + 860221
7   chromedriver                        0x00000001028d8e76 chromedriver + 970358
8   chromedriver                        0x00000001028bdde3 chromedriver + 859619
9   chromedriver                        0x000000010288bd7f chromedriver + 65

In [7]:
## Create initial csv
df_mercadona.to_csv("../data/mercadona.csv",index=False)

In [8]:
df_mercadona.to_csv("../data/mercadona.csv", mode='a', index=False, header=False)
mercadona = pd.read_csv("../data/mercadona.csv")
print("Appended",today)
mercadona.tail()

Appended 2023-06-21 19:06:53.250287


,Group,Category,Name,Size,Price,Image,Shop,Date
703,Agua y refrescos,Refresco de té y sin gas,Refresco piña-coco Gold Spring cero sin gas,Botella 2 L,"1,30 € /ud.",https://prod-mercadona.imgix.net/images/97d759...,Mercadona,2023-06-21 19:06:53.250287
704,Agua y refrescos,Refresco de té y sin gas,Agua de coco Chaokoh 100% natural,Brick 1 L,"2,60 € /ud.",https://prod-mercadona.imgix.net/images/97d759...,Mercadona,2023-06-21 19:06:53.250287
705,Agua y refrescos,Refresco de té y sin gas,Refresco sandía Gold Spring cero sin gas,Botella 2 L,"1,30 € /ud.",https://prod-mercadona.imgix.net/images/97d759...,Mercadona,2023-06-21 19:06:53.250287
706,Agua y refrescos,Refresco de té y sin gas,Refresco Caribe Gold Spring cero sin gas,Botella 2 L,"1,30 € /ud.",https://prod-mercadona.imgix.net/images/97d759...,Mercadona,2023-06-21 19:06:53.250287
707,Agua y refrescos,Refresco de té y sin gas,Limonada Hacendado light sin gas,4 botellas x 330 ml,"1,50 € /pack",https://prod-mercadona.imgix.net/images/97d759...,Mercadona,2023-06-21 19:06:53.250287
